In [1]:
!pip install bs4

    100% |████████████████████████████████| 92kB 1.4MB/s ta 0:00:011
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [ ]:
#SETN作者欄位清理方法，要放author[0]
def setn_author_etl(author_text):
    author = []
    #將／XX報導XX字串清除
    author_text = re.sub(r"／.*報導.*", "", author_text)
    #將／XX特稿XX字串清除
    author_text = re.sub(r"／.*特稿.*", "", author_text)
    #將記者字串清除
    author_text = re.sub(r"記者", "", author_text)
    #將文／字串清除
    author_text = re.sub(r"文／", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    else:
        author.append(author_text)
    return print(author)


In [1]:
#不包含作者欄位清理版本
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import json
import re

page = 1
json_data = []
while True:
    page_url = "https://www.setn.com/ViewAll.aspx?PageGroupID=6&p=" + str (page)
    print ("Processing:", page_url)
    
    response = urlopen (page_url)
    bs = BeautifulSoup (response)
    titles = bs.find_all ("h3", "view-li-title")

    if len (titles) == 0:
        break
        
    #times = 0
    for t in titles:
        
        #if times==2:
        #    break
        #times = times + 1
        
        print("新聞來源:", "SETN")
        
        news_url = "https://www.setn.com" + t.find ("a")["href"]
        print("新聞網址:", news_url)
        
        req = requests.get (news_url)
        bs = BeautifulSoup (req.text, 'html.parser')
        
        title = bs.find('h1', class_ = "news-title-3").text
        title = re.sub(r"\u3000", " ", title)
        print("新聞標題:", title)
        
        date = bs.find("time", class_ = "page-date").text.strip()
        print("發佈時間:", date)
        
        author = []
        a_divs = bs.find("div", id="Content1")
        first_p = a_divs.find("p")
        author_text = first_p.text
        #找第一段有沒有含"／"，如果沒有給予空字串
        if author_text.find("／") == -1:
            author_text = ""
        #如果作者是空字串，給予空list
        if author_text == "":
            author = []
        else:
            author.append(author_text)
        print("作者:", author)
             
        c_divs = bs.find('div', itemprop='articleBody')
        content_all = c_divs.find_all ("p")
        content = ""
        for c in content_all:
            if c.attrs == {}:
                content = content + c.text
                #print(c.text)
                
                c_divs = bs.find('div', itemprop='articleBody')
        content_all = c_divs.find_all ("p")
        content = ""
        for c in content_all:
            if c.attrs == {}:
                content = content + c.text
                #print(c.text)
        print("內文:", content)
        
        keyword = []
        k_divs = bs.find("div", class_="keyword page-keyword-area")
        k_strong = k_divs.find_all("strong")
        for k in k_strong:
            keyword_text = k.text
            keyword.append(keyword_text)
        print("關鍵字:", keyword)
        
        img_p = bs.find("p", style = "text-align: center;")
        img_url = img_p.find("img")["src"]
        print("圖片網址:", img_url)
        
        print("\n")
      
        time.sleep (1)
        
        j = {"source": "SETN" ,"url": news_url, "title": title, "release_datetime": date, "author": author, "content": content, 
             "keyword": keyword, "img_url": img_url}
        json_data.append(j)
    time.sleep (2)
    page = page + 1

KeyboardInterrupt: 

In [ ]:
for i in range(0, len(json_data)):  
    if json_data[i]["author"] == []:
        author = []
        print(author)
    else:
        setn_author_etl(json_data[i]["author"][0])

In [44]:
#包含作者欄位清理版本
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import json
import re

page = 1
json_data = []
while True:
    page_url = "https://www.setn.com/ViewAll.aspx?PageGroupID=6&p=" + str (page)
    print ("Processing:", page_url)
    
    response = urlopen (page_url)
    bs = BeautifulSoup (response)
    titles = bs.find_all ("h3", "view-li-title")

    if len (titles) == 0:
        break
        
    #times = 0
    for t in titles:
        
        #if times==2:
        #    break
        #times = times + 1
        print("新聞來源:", "SETN")
        
        news_url = "https://www.setn.com" + t.find ("a")["href"]
        print("新聞網址:", news_url)
        
        req = requests.get (news_url)
        bs = BeautifulSoup (req.text, 'html.parser')
        
        title = bs.find('h1', class_ = "news-title-3").text
        title = re.sub(r"\u3000", " ", title)
        print("新聞標題:", title)
    
        date = bs.find("time", class_ = "page-date").text.strip()
        print("發佈時間:", date)
        
        author = []
        a_divs = bs.find("div", id="Content1")
        first_p = a_divs.find("p")
        author_text = first_p.text
        #找第一段有沒有含"／"，如果沒有給予空字串
        if author_text.find("／") == -1:
            author_text = ""
        #將／XX報導XX字串清除
        author_text = re.sub(r"／.*報導.*", "", author_text)
        #將／XX特稿XX字串清除
        author_text = re.sub(r"／.*特稿.*", "", author_text)
        #將記者字串清除
        author_text = re.sub(r"記者", "", author_text)
        #將文／字串清除
        author_text = re.sub(r"文／", "", author_text)
        #如果有多位作者，使用、斷開，並一一取出放進list
        if author_text.find("、") != -1:
            author_text = author_text.split("、")
            for a in author_text:
                author.append(a)
        #如果作者是空字串，給予空list
        elif author_text == "":
            author = []
        else:
            author.append(author_text)
        print("作者:", author)
       
        c_divs = bs.find('div', itemprop='articleBody')
        content_all = c_divs.find_all ("p")
        content = ""
        for c in content_all:
            if c.attrs == {}:
                content = content + c.text
                #print(c.text)
        print("內文:", content)
        
        keyword = []
        k_divs = bs.find("div", class_="keyword page-keyword-area")
        k_strong = k_divs.find_all("strong")
        for k in k_strong:
            keyword_text = k.text
            keyword.append(keyword_text)
        print("關鍵字:", keyword)
        
        img_p = bs.find("p", style = "text-align: center;")
        img_url = img_p.find("img")["src"]
        print("圖片網址:", img_url)
        
        print("\n")
        
        time.sleep (1)
        
        j = {"source": "SETN" ,"url": news_url, "title": title, "release_datetime": date, "author": author, "content": content, 
             "keyword": keyword, "img_url": img_url}
        json_data.append(j)
    time.sleep (2)
    page = page + 1
    
f = open("news_setn.json", "w", encoding="utf-8")
f.write(json.dumps(result))
f.close()


Processing: https://www.setn.com/ViewAll.aspx?PageGroupID=6&p=1
新聞網址: https://www.setn.com/News.aspx?NewsID=470662
新聞標題: 「嚴肅問題自備音樂」 卓榮泰記者會幽默回應笑翻全場
發佈時間: 2018/12/14 15:47:00
作者: ['記者林仕祥／台北報導']
內文: 記者林仕祥／台北報導行政院祕書長卓榮泰今（14）日在黨內中生代的陪同下，登記參選民進黨黨主席，媒體詢問日前卓榮泰嗆民進黨立委段宜康「出來選黨主席」一事時，現場突然響起手機音樂，記者笑問「為何自備配樂」，卓榮泰也幽默神回，「問到比較嚴肅的問題就會出現」。桃園市長鄭文燦昨晚在臉書以「中生代謙卑反省、共同承擔責任」為題發文指出，黨內跨派系中生代力推卓榮泰參選黨主席，文末署名「鄭文燦、林佳龍、潘孟安、林智堅、黃偉哲、翁章梁及陳其邁」共推，卓榮泰也在今天下午前往登記，不過台中市長林佳龍、屏東縣長潘孟安並沒有出席，鄭文燦說，他們兩人一人出國、一人在地方有要務。登記結束後，卓榮泰在電梯前接受媒體詢問，記者詢問，卓榮泰日前曾嗆過段宜康「出來選黨主席」，是否當時就有參選的意願？卓榮泰強調，完全沒有，他向段宜康喊話的第二天，就曾講過，現在台灣社會有絕大多數的民眾，希望看到的是民進黨的自我檢討，民進黨要讓民眾得到這樣的訊息，就不要再去檢討別人，「我想段委員也清楚這件事情，這件事是就事論事，沒有針對他個人」。不過卓榮泰在回答的過程中，現場竟然響起手機音樂，被虧說自備配樂，卓榮泰說，「問到比較嚴肅的問題，就會出現」，幽默的回答讓現場記者們紛紛大笑。據了解，這首歌是日劇《法醫女王》的主題曲《Lemon》。卓榮泰最後更強調，他跟段宜康沒有誤會，「一定會見面」。 
關鍵字: ['段宜康', '民進黨主席', '卓榮泰', '法醫女王']
圖片網址: https://attach.setn.com/newsimages/2018/12/14/1687366-XXL.jpg


新聞網址: https://www.setn.com/News.aspx?NewsID=470639
新聞標題: 推卓榮泰任黨主席！他：民進黨內部再無「拔蔡」議題
發佈時間: 2018/12/14 15:40:00
作者: ['政治中心／綜合

KeyboardInterrupt: 

In [45]:
json_data

[{'url': 'https://www.setn.com/News.aspx?NewsID=470662',
  'title': '「嚴肅問題自備音樂」 卓榮泰記者會幽默回應笑翻全場',
  'release_datetime': '2018/12/14 15:47:00',
  'author': ['記者林仕祥／台北報導'],
  'content': '記者林仕祥／台北報導行政院祕書長卓榮泰今（14）日在黨內中生代的陪同下，登記參選民進黨黨主席，媒體詢問日前卓榮泰嗆民進黨立委段宜康「出來選黨主席」一事時，現場突然響起手機音樂，記者笑問「為何自備配樂」，卓榮泰也幽默神回，「問到比較嚴肅的問題就會出現」。桃園市長鄭文燦昨晚在臉書以「中生代謙卑反省、共同承擔責任」為題發文指出，黨內跨派系中生代力推卓榮泰參選黨主席，文末署名「鄭文燦、林佳龍、潘孟安、林智堅、黃偉哲、翁章梁及陳其邁」共推，卓榮泰也在今天下午前往登記，不過台中市長林佳龍、屏東縣長潘孟安並沒有出席，鄭文燦說，他們兩人一人出國、一人在地方有要務。登記結束後，卓榮泰在電梯前接受媒體詢問，記者詢問，卓榮泰日前曾嗆過段宜康「出來選黨主席」，是否當時就有參選的意願？卓榮泰強調，完全沒有，他向段宜康喊話的第二天，就曾講過，現在台灣社會有絕大多數的民眾，希望看到的是民進黨的自我檢討，民進黨要讓民眾得到這樣的訊息，就不要再去檢討別人，「我想段委員也清楚這件事情，這件事是就事論事，沒有針對他個人」。不過卓榮泰在回答的過程中，現場竟然響起手機音樂，被虧說自備配樂，卓榮泰說，「問到比較嚴肅的問題，就會出現」，幽默的回答讓現場記者們紛紛大笑。據了解，這首歌是日劇《法醫女王》的主題曲《Lemon》。卓榮泰最後更強調，他跟段宜康沒有誤會，「一定會見面」。\xa0',
  'keyword': ['段宜康', '民進黨主席', '卓榮泰', '法醫女王'],
  'img_url': 'https://attach.setn.com/newsimages/2018/12/14/1687366-XXL.jpg'},
 {'url': 'https://www.setn.com/News.aspx?NewsID=470639',
  'title': '推卓榮泰任黨主席！他：民進黨內部再無「拔蔡」議題',
  'release_dat